In [121]:
import graph_tool.all as gt
import os
import numpy as np

In [122]:
np.random.geometric(0.5)

3

In [123]:
data_dict = {
    'offline': {
        "school_friendship": [
            "spanish_highschools__1",
            "spanish_highschools__2",
            "spanish_highschools__6",
            "spanish_highschools__11_10",
            "spanish_highschools__11_9",
        ],
        "school_contact": [
            "sp_high_school_new__2011",
            "sp_high_school_new__2012",
            "sp_primary_school",
        ],
        'email': [
            "email_enron",
            "uni_email",
            "email_eu",
            "dnc"
        ]
    },
    'online': {
        'facebook': [
            "facebook_wall",
            "facebook_organizations__L1",
            "facebook_organizations__L2",
            "facebook_organizations__M1",
            "ego_social__facebook_107",
            "ego_social__facebook_1912",
            "ego_social__facebook_combined",
        ],
        'google_plus': [
            "ego_social__gplus_101133961721621664586",
            "ego_social__gplus_100500197140377336562",
            "ego_social__gplus_101133961721621664586",
            "ego_social__gplus_114336431216099933033",
        ],
        'twitter': [
            "twitter_15m",
            "twitter",
        ],
    },
}

In [124]:
def duplicate_graph(g, k):
    """
    Creates a new graph by combining `k` disjoint copies of graph `g`.
    """
    combined = gt.Graph()
    vertex_maps = []

    for _ in range(k):
        g_copy = g.copy()
        iterator = combined.add_vertex(g_copy.num_vertices())
        node_map = {v: next(iterator) for v in g_copy.vertices()}
        # Add edges with correct offset
        for e in g_copy.edges():
            src = int(e.source())
            tgt = int(e.target())
            combined.add_edge(node_map[src], node_map[tgt])
    
    return combined

In [125]:
import graph_tool.all as gt
import random
def extract_subgraph(g, node_set):
    vfilt = g.new_vertex_property("bool")
    for v in node_set:
        vfilt[int(v)] = True  # convert Vertex to int (vertex index)
    gv = gt.GraphView(g, vfilt=vfilt)
    return gt.Graph(gv, prune=True)
    
def forest_fire_sample(g, sample_size=500, pf=0.5):
    """
    Forest Fire Sampling on a graph-tool Graph.
    - g: graph-tool Graph
    - sample_size: desired number of nodes in sample
    - pf: forward burning probability
    """

    visited = set()
    not_visited = set(g.vertices())
    to_visit = []

    ego_count = 0
    while len(visited) < sample_size:
        if not to_visit or ego_count > sample_size:
            to_visit = [np.random.choice(list(not_visited))]
            ego_count = 0
        
        current = to_visit.pop()
        if current in visited:
            ego_count += 1
            continue

        visited.add(current)
        not_visited.remove(current)

        neighbors = list(current.out_neighbors())
        scaled_num_neighbors = max(min(len(neighbors), 5), int(len(neighbors) * sample_size / g.num_vertices()))
        num_to_burn = min(scaled_num_neighbors, np.random.geometric(pf))
        num_samples = scaled_num_neighbors - num_to_burn
        neighbors_sample = random.sample(neighbors, num_samples)
        to_visit.extend(neighbors_sample)

    return extract_subgraph(g, visited)

def summary(g):
    return ", \t ".join([
        f"nodes: {g.num_vertices():.2f}",
        f"edges: {g.num_edges():.2f}",
        f"avg_degree: {2 * g.num_edges() / g.num_vertices():.2f}",
        f"avg_clustering: {gt.global_clustering(g)[0]:.2f}",
    ])

In [126]:
def export_graph(g, full_name, g_type, path):
    seen = set()
    filepath = f"{path}/{g_type}/{full_name}.edge"
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w") as f:
        for e in g.edges():
            u, v = int(e.source()), int(e.target())
            edge = tuple(sorted((u, v)))
            if edge not in seen:
                seen.add(edge)
                f.write(f"{edge[0]} {edge[1]}\n")

In [127]:
source_dir = os.path.join("..", "..", "DATA", "netzschleuder")
target_dir = os.path.join("..", "..", "DATA", "processed")

network_list = [net for off_on in data_dict.values() for category in off_on.values() for net in category]
# network_list = ['sp_high_school_new__2011']


for network in network_list:
    # Load the graph
    g = gt.load_graph(f"{source_dir}/{network}.gt")

    # Get the number of vertices and edges
    num_nodes = g.num_vertices()
    num_edges = g.num_edges()

    g.set_directed(False)
    gt.remove_parallel_edges(g)
    gt.remove_self_loops(g)    

    # Print the number of vertices and edges
    print(f"\n Graph: {network}")
    print("\t Original: \t", summary(g))
    export_graph(g, network, "original", target_dir)

    sample_size = 400
    if num_nodes < sample_size:
        g_new = duplicate_graph(g, int(np.ceil(sample_size / num_nodes)))
        sample = forest_fire_sample(g_new, sample_size=sample_size, pf=0.5)  
    else:
        sample = forest_fire_sample(g, sample_size=sample_size, pf=0.5)    
    print("\t Sampled: \t", summary(sample))
    export_graph(sample, network, "sample", target_dir)





 Graph: spanish_highschools__1
	 Original: 	 nodes: 409.00, 	 edges: 6509.00, 	 avg_degree: 31.83, 	 avg_clustering: 0.34
	 Sampled: 	 nodes: 400.00, 	 edges: 6387.00, 	 avg_degree: 31.93, 	 avg_clustering: 0.34

 Graph: spanish_highschools__2
	 Original: 	 nodes: 238.00, 	 edges: 2889.00, 	 avg_degree: 24.28, 	 avg_clustering: 0.47
	 Sampled: 	 nodes: 400.00, 	 edges: 4179.00, 	 avg_degree: 20.89, 	 avg_clustering: 0.49

 Graph: spanish_highschools__6
	 Original: 	 nodes: 534.00, 	 edges: 9527.00, 	 avg_degree: 35.68, 	 avg_clustering: 0.45
	 Sampled: 	 nodes: 400.00, 	 edges: 6208.00, 	 avg_degree: 31.04, 	 avg_clustering: 0.46

 Graph: spanish_highschools__11_10
	 Original: 	 nodes: 458.00, 	 edges: 5866.00, 	 avg_degree: 25.62, 	 avg_clustering: 0.39
	 Sampled: 	 nodes: 400.00, 	 edges: 5121.00, 	 avg_degree: 25.61, 	 avg_clustering: 0.41

 Graph: spanish_highschools__11_9
	 Original: 	 nodes: 515.00, 	 edges: 5027.00, 	 avg_degree: 19.52, 	 avg_clustering: 0.34
	 Sampled: 	 nodes

In [128]:
# gt.graph_draw(g, output_size=(600, 600), bg_color=(255, 255, 255, 255))
# gt.graph_draw(sample, output_size=(600, 600), bg_color=(255, 255, 255, 255))
